# Lab 1: Environment Setup (Foundry Local and Azure Foundry)

**Purpose:** Configure both the local and cloud environments required for the hybrid architecture.

## Overview

In this lab, we'll set up:
- Azure Foundry Local for on-device LLM inference
- Azure AI Foundry project for cloud-based LLM capabilities
- Required Python libraries and environment variables

This creates the foundation for building a hybrid AI chatbot that can route queries between local and cloud models.

## Step 1.1: Install Azure Foundry Local

Azure Foundry Local allows you to run LLMs directly on your device for fast, private inference.

### Installation Instructions:

**Windows:**
```bash
winget install Microsoft.FoundryLocal
```

**macOS:**
```bash
brew tap microsoft/foundrylocal && brew install foundrylocal
```

After installation, verify by running the following in a terminal:

In [ ]:
# Verify Foundry Local installation
import subprocess
try:
    result = subprocess.run(['foundry', '--version'], capture_output=True, text=True)
    print(f"Foundry Local version: {result.stdout.strip()}")
    print("✅ Foundry Local is installed successfully")
except FileNotFoundError:
    print("❌ Foundry Local not found. Please install it using the instructions above.")

## Step 1.2: Download and Start a Local Model

We'll use a lightweight model like `phi-3.5-mini` that's optimized for local execution. 

*You can choose another lightweight model. For a list of models, please run `foundry model list` in the terminal before executing the command below.*

**Run this command in your terminal (not in the notebook):**
```bash
foundry model run phi-3.5-mini
```

This will:
1. Download the model (may take several minutes for first run)
2. Start a local service hosting the model
3. Provide an endpoint for API calls

Keep this terminal open throughout the workshop!

In [ ]:
from foundry_local import FoundryLocalManager

# Initialize and optionally bootstrap with a model
manager = FoundryLocalManager(alias_or_model_id=None, bootstrap=True)

print("Is service running: ", manager.is_service_running())
print("Service URI: ", manager.service_uri)
print("Service Endpoint: ", manager.endpoint)

In [ ]:
import os
import sys

# Add parent directory to path for module imports
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)


Let's update our `.env` file with the endpoint for Azure Foundry Local.

In [ ]:
from dotenv import find_dotenv, load_dotenv, set_key

load_dotenv(find_dotenv(".env"))

# Update a variable
key_to_update = "LOCAL_MODEL_ENDPOINT"
new_value = manager.service_uri

# Update the .env file
set_key(os.path.join(parent_dir, ".env"), key_to_update, new_value)

In [ ]:
# Check if local model service is running
import requests

try:
    # Common local endpoint for Foundry Local
    local_endpoint = manager.service_uri # Adjust if your Foundry Local uses a different port
    response = requests.get(f"{local_endpoint}/openai/status", timeout=5)
    if response.status_code == 200:
        print("✅ Local model service is running")
        print(f"Endpoint: {local_endpoint}")
    else:
        print(f"⚠️  Local service responded with status {response.status_code}")
except requests.exceptions.RequestException as e:
    print("❌ Local model service is not running or not accessible")
    print("Please run 'foundry model run phi-3.5-mini' in a terminal")

## Step 1.3: Set Up Azure AI Foundry Project

### Prerequisites:
1. An Azure subscription with access to Azure OpenAI services
2. An Azure AI Foundry project created
3. A deployed chat model (e.g., GPT-4, GPT-3.5-turbo)

### Required Information:
If using Azure AI Foundry Agents
- Azure AI Foundry endpoint URL
- Credential

If using Azure OpenAI direct
- Azure OpenAI endpoint URL
- API key
- Version
- Deployment name

If you don't have these set up yet, follow the Azure AI Foundry documentation to create a project and deploy a model.

## Step 1.4: Configure Environment Variables

Set up the configuration for both local and cloud connections:

In [ ]:
import os

from azure.identity import DefaultAzureCredential
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(".env"))

# Local model configuration
LOCAL_ENDPOINT = os.environ["LOCAL_MODEL_ENDPOINT"]  # Adjust if your Foundry Local uses a different port
LOCAL_MODEL_ALIAS = os.environ["LOCAL_MODEL_NAME"]

AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_KEY = os.environ["AZURE_OPENAI_KEY"]
AZURE_OPENAI_DEPLOYMENT = os.environ["AZURE_DEPLOYMENT_NAME"]
AZURE_OPENAI_API_VERSION = os.environ["AZURE_OPENAI_API_VERSION"]

print("✅ Environment variables configured")
print(f"Local endpoint: {LOCAL_ENDPOINT}")
print(f"Azure endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Azure deployment: {AZURE_OPENAI_DEPLOYMENT}")
print(f"Azure API version: {AZURE_OPENAI_API_VERSION}")

## Step 1.5: Install Required Libraries

Install the Python packages needed for the workshop:

In [ ]:
# # Install required packages - This is only needed if you haven't installed the packages yet
# !pip install openai azure-ai-ml streamlit requests python-dotenv

## Step 1.6: Test Connectivity

Let's verify that we can connect to both local and cloud services:

In [ ]:
from openai import AzureOpenAI
import requests

# Test local connectivity
print("Testing local model connectivity...")
try:
    # For local Foundry service, we'll use a simple health check
    response = requests.get(f"{LOCAL_ENDPOINT}/openai/status", timeout=10)
    if response.status_code == 200:
        print("✅ Local model service is accessible")
    else:
        print(f"⚠️  Local service responded with status {response.status_code}")
except Exception as e:
    print(f"❌ Local model connection failed: {e}")

# Test Azure connectivity
print("\nTesting Azure OpenAI connectivity...")
try:
    # Initialize Azure OpenAI client
    azure_client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        base_url=f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{AZURE_OPENAI_DEPLOYMENT}",
        api_version=AZURE_OPENAI_API_VERSION
    )
    
    # Test with a simple completion
    response = azure_client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[{"role": "user", "content": "Hello, Azure!"}],
        max_tokens=10
    )
    print("✅ Azure OpenAI connection successful")
    print(f"Test response: {response.choices[0].message.content}")
except Exception as e:
    print(f"❌ Azure OpenAI connection failed: {e}")
    print("Please check your endpoint, API key, and deployment name")

## Step 1.7: Save Configuration

Create a configuration file for use in subsequent labs:

In [ ]:
# Create a config file for the workshop
config_content = f'''
# Hybrid LLM Router Workshop Configuration

# Local Model Configuration
LOCAL_ENDPOINT={LOCAL_ENDPOINT}
LOCAL_MODEL_ALIAS={LOCAL_MODEL_ALIAS}

# Azure OpenAI Configuration
AZURE_OPENAI_ENDPOINT={AZURE_OPENAI_ENDPOINT}
AZURE_OPENAI_KEY={AZURE_OPENAI_KEY}
AZURE_OPENAI_DEPLOYMENT={AZURE_OPENAI_DEPLOYMENT}
AZURE_OPENAI_API_VERSION={AZURE_OPENAI_API_VERSION}
'''

# with open('.env', 'w') as f:
#     f.write(config_content)

print("✅ Configuration saved to .env file")
print("This file will be used by subsequent labs")

## 🎉 Lab 1 Complete!

You have successfully:
- ✅ Installed Azure Foundry Local
- ✅ Started a local model service
- ✅ Configured Azure AI Foundry connection
- ✅ Installed required Python libraries
- ✅ Verified connectivity to both services
- ✅ Saved configuration for future labs

### Next Steps:
- Proceed to Lab 2 to test the on-device LLM
- Keep your local model service running throughout the workshop
- If you encounter issues, check the troubleshooting section in the workshop documentation

### Troubleshooting:
- If the local model isn't starting, ensure you have enough RAM and disk space
- If Azure connection fails, verify your subscription has access to Azure OpenAI
- Check firewall settings if you can't connect to localhost:8080